In [17]:
# Importing libraries and paths

import pandas as pd
import gzip
import ast
from datetime import datetime
import calendar 
import re
# from textblob import TextBlob

# Paths

path1 = 'steam_games.json.gz'
path2 = 'user_reviews.json.gz'
path3 = 'users_items.json.gz'

# Creating a function to read the paths

def read_path(file):
    with gzip.open(file, 'rt', encoding='utf-8') as myfile:
        return [ast.literal_eval(line.strip()) for line in myfile]


# We generated the function to unnest the 'items' and 'reviews' columns
def unnest_items(df):
    # Explode the 'items' column to create a new row for each element in the list
    items = df['items'].explode()

    # Drop the original 'items' column from the DataFrame
    df = df.drop('items', axis=1)

    # Join the exploded 'items' Series as new columns in the DataFrame
    df = df.join(items.apply(pd.Series))

    # Return the modified DataFrame
    return df


In [29]:
def read_path(file):
    with gzip.open(file, 'rt', encoding='utf-8') as myfile:
        return [ast.literal_eval(line.strip()) for line in myfile]

items = read_path(path3)
df = pd.DataFrame(items)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None


In [30]:
#   Let's read all the datasets and save it in files

# Reading the dataset from 'steam_games.json.gz' ↓↓↓
with gzip.open(path1, 'rt', encoding='utf-8') as file:
    df_games = pd.read_json(file, lines=True)

df_games.to_csv("Games.csv")
# Saving ↑↑↑ our dataframe 'Games' to a csv file 

# Reading datasets 
reviews = read_path(path2)
items = read_path(path3)

# Transforming datasets to dataframes ↓:
df_reviews = pd.DataFrame(reviews)
df_items = pd.DataFrame(items)


# Saving dataframes in csv and parquet files respectively ↓:
df_reviews.to_csv("Reviews.csv")
df_items.to_parquet("Items.parquet")

# Why did I choose Parquet? This is related to the structure of the dataset itself.

# Note: this cell might take some time to finish the task.

In [ ]:
# We read an element of the dataset to see what's inside 


HOla este es otro mensaje de prueba...
